In [2]:
pip install nltk transformers torch tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [1]:
import wandb

wandb.login(key="0c5f368f1f51fd942ec7bb3a1c74efb7bdc832d6")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mshoaibvohra (mshoaibvohra-habib-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
import nltk
nltk.download('punkt', download_dir='/content/nltk_data')
nltk.download('wordnet', download_dir='/content/nltk_data')
nltk.data.path.append('/content/nltk_data')
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')  # Optional, for better synonym support


nltk.download("punkt_tab")

[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /content/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [7]:
import json
import torch
import nltk
import random
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification

nltk.download("punkt")

# Load tweet data (X)
file_path = "/content/EXIST2025_training_translated_en.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Load label data (Y)
label_path = "/content/EXIST2025_training_task1_3_gold_soft.json"
with open(label_path, "r", encoding="utf-8") as f:
    labels = json.load(f)

# Map from tweet ID to label
label_map = {item["id"]: item for item in labels}

# Load Pretrained DistilBERT
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# AEDA augmentation
def aeda_augment(text, punct_list=[".", ",", "!", "?", ";", ":"]):
    words = text.split()
    if len(words) < 3:
        return text
    n_insertions = random.randint(1, max(1, len(words) // 3))
    positions = random.sample(range(len(words)), n_insertions)
    positions.sort()
    for i, pos in enumerate(positions):
        punct = random.choice(punct_list)
        words.insert(pos + i, punct)
    return ' '.join(words)

# Sentence utility functions
def extract_units(text):
    sentences = nltk.sent_tokenize(text)
    return sentences if len(sentences) > 1 else [text]

def sift_sentences(sentences):
    return [s for s in sentences if len(s.split()) > 3]

# Augmentation loop
augmented_data = {}
augmented_labels = []
augmented_count = 0
AUGMENT_LIMIT = 3500
used_ids = []

shuffled_items = list(data.items())
random.shuffle(shuffled_items)

for key, value in tqdm(shuffled_items, desc="AEDA Augmenting Tweets"):
    if augmented_count >= AUGMENT_LIMIT:
        break

    original_id = value["id_EXIST"]
    if original_id in used_ids:
        continue

    original_text = value["tweet"]
    extracted_units = extract_units(original_text)
    filtered_units = sift_sentences(extracted_units)

    for i, unit in enumerate(filtered_units):
        if augmented_count >= AUGMENT_LIMIT:
            break

        aeda_unit = aeda_augment(unit)
        aug_key = f"{key}_AEDA_{i}"
        aug_id = f"{original_id}_AEDA_{i}"

        augmented_data[aug_key] = {
            **value,
            "id_EXIST": aug_id,
            "tweet": aeda_unit,
            "split": "AUG_EN"
        }

        if original_id in label_map:
            original_label = label_map[original_id]
            new_label = {
                "test_case": original_label["test_case"],
                "id": aug_id,
                "value": original_label["value"]
            }
            augmented_labels.append(new_label)
            augmented_count += 1

    used_ids.append(original_id)

# Save outputs
data.update(augmented_data)
all_labels = labels + augmented_labels

with open("EXIST2025_training_augmented_AEDA_en.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

with open("EXIST2025_training_augmented_gold_AEDA_en.json", "w", encoding="utf-8") as f:
    json.dump(all_labels, f, ensure_ascii=False, indent=4)

with open("used_tweet_ids_AEDA.json", "w") as f:
    json.dump(used_ids, f, indent=2)

print(f"\n✅ AEDA Augmentation Complete: {augmented_count} new samples added.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
AEDA Augmenting Tweets:  32%|███▏      | 2191/6920 [00:00<00:00, 7206.76it/s] 



✅ AEDA Augmentation Complete: 3500 new samples added.
